In [1]:
## Code for making Figure 7 and appendix rate maps

In [2]:
import os
#os.chdir('path_to_where_files_are')
#os.getcwd()

In [3]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
import time
import torch
from scipy.stats import pearsonr
#import pickle5 as pickle
import sys
from scipy.spatial import distance
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter1d, binary_closing
from scipy import stats
from scipy import signal
import time

sys.path.append(os.path.join(os.getcwd(), 'NeuralLVM'))
import NeuralLVM.training
from NeuralLVM.training import *


device = "cuda" if torch.cuda.is_available() else "cpu"
print('Running on', device)

skip = 3
#grid_cell_data_test = np.loadtxt("spikegrid100_5000_e.csv", delimiter=",")[::1,::skip]
#grid_cell_data_train = np.loadtxt("spikegrid100_5000_e.csv", delimiter=",")[::1,::skip]
grid_cell_data_test = np.loadtxt("spikegrid100_5000_e.csv", delimiter=",")[::1,::skip]
grid_cell_data_train = np.loadtxt("spikegrid100_5000_e.csv", delimiter=",")[::1,::skip]
traj = np.loadtxt("trj_grid_e.csv", delimiter=",")[::skip,:]

#grid_cell_data = np.load('grid_cell_data_100ms.npy', allow_pickle=True)
#rate_map_data = np.load('rate_map_data.npy', allow_pickle=True)

Running on cpu
Running on cpu


In [6]:
## Load best bump model
num_neuron_train = 100
num_neuron_test = 100
num_ensemble = 1
latent_dim = 2
global_seed = 42
index = 4
feature_type = 'bump'
model = LatentVariableModel(
      num_neuron_train=num_neuron_train,
      num_neuron_test=num_neuron_test,
      num_hidden=256,
      num_ensemble=num_ensemble,
      latent_dim=latent_dim,
      seed=global_seed + index,
      tuning_width=5.0, #10.0
      nonlinearity='exp',
      kernel_size=9, #9
      feature_type=feature_type,
      num_feature_basis=50, #10
  ).to(device)
trainer = Trainer(
      model=model,
      #data_train=grid_cell_data.item()['y_train'],
      data_train=grid_cell_data_test,
      data_test=grid_cell_data_train,
      #neurons_train_ind=grid_cell_data.item()['neurons_train_ind'],
      #neurons_train_ind=np.array([0,1,2]),
      neurons_train_ind=np.arange(0, 100, 1),
      mode='full',
      z_train=None,
      z_test=None,
      num_steps=30000,
      batch_size=64,
      seed=global_seed + index,
      learning_rate=3e-3,
      num_worse=10, # if loss doesn't improve X times, stop.
      weight_entropy=1e-3
  )

#model.load_state_dict(torch.load('model.pth'))

In [ ]:
#print(model)
trainer.train()

run=1000, running_loss=-1.9403e+03, negLLH_train=-4.9158e+00, negLLH_test=-4.3772e+00, KL_normal=1.3072e+00, Slowness_loss=9.6688e-01, corr=0.448929, H=-9.5367e-07, time=5.10
run=2000, running_loss=-5.4816e+03, negLLH_train=-7.2154e+00, negLLH_test=-5.9332e+00, KL_normal=2.4760e+00, Slowness_loss=8.8504e-01, corr=0.573642, H=-9.5367e-07, time=10.67
run=3000, running_loss=-6.5048e+03, negLLH_train=-7.8461e+00, negLLH_test=-5.2071e+00, KL_normal=2.5042e+00, Slowness_loss=8.1352e-01, corr=0.564344, H=-9.5367e-07, time=16.18
run=4000, running_loss=-6.9715e+03, negLLH_train=-7.9865e+00, negLLH_test=-6.3816e+00, KL_normal=2.5811e+00, Slowness_loss=7.7125e-01, corr=0.593401, H=-9.5367e-07, time=22.15
run=5000, running_loss=-7.1188e+03, negLLH_train=-7.9966e+00, negLLH_test=-6.4665e+00, KL_normal=2.5677e+00, Slowness_loss=7.7884e-01, corr=0.599844, H=-9.5367e-07, time=28.63
run=6000, running_loss=-7.1183e+03, negLLH_train=-8.0107e+00, negLLH_test=-6.5965e+00, KL_normal=2.6893e+00, Slowness_los

In [ ]:
_, _, z_, _, _ = model(torch.tensor(grid_cell_data_train, dtype=torch.float32, device=device)[trainer.neurons_train_ind], z=None)
print(z_.shape)
print(z_)
z_mu = z_[0,:,0,:]
print(z_mu.shape)
z_mu = z_mu.to('cpu').detach().numpy().copy()
print(z_mu.shape[0])
plt.figure(figsize=(15,5))
plt.scatter(np.arange(z_mu.shape[0]), z_mu[:,0], marker = ".", color = "k", s = 100)
#for i in range(len(thetas)-1):
  #x1, x2 = thetas[i], thetas[i+1]
  #if np.abs(x1-x2) < np.pi: plt.plot([i, i+1], [x1, x2], "k-")
plt.xlabel('time (a.u.)')
plt.ylabel(r'$\theta$')
plt.show()
plt.figure(figsize=(15,5))
plt.scatter(np.arange(z_mu.shape[0]), z_mu[:,1], marker = ".", color = "k", s = 100)
#for i in range(len(thetas)-1):
  #x1, x2 = thetas[i], thetas[i+1]
  #if np.abs(x1-x2) < np.pi: plt.plot([i, i+1], [x1, x2], "k-")
plt.xlabel('time (a.u.)')
plt.ylabel(r'$\theta$')
plt.show()

In [ ]:
threshold = .0
for t in range(30):
    thetas_b = np.copy(z_mu)
    for i in range(z_mu.shape[0]-1):
        #print(thetas.shape)
        if thetas_b[i+1,0] - thetas_b[i,0] > 2*np.pi-threshold - t/15:
            thetas_b[i+1:,0] -= 2*np.pi#qz_est_norm[i+1,0] - qz_est_norm[i,0]
        if thetas_b[i+1,1] - thetas_b[i,1] > 2*np.pi-threshold - t/15:
            thetas_b[i+1:,1] -= 2*np.pi#qz_est_norm[i+1,1] - qz_est_norm[i,1]
        if thetas_b[i+1,0] - thetas_b[i,0] < - 2*np.pi+threshold + t/15:
            thetas_b[i+1:,0] += 2*np.pi#qz_est_norm[i,0] - qz_est_norm[i+1,0]
        if thetas_b[i+1,1] - thetas_b[i,1] < - 2*np.pi+threshold + t/15:
            thetas_b[i+1:,1] += 2*np.pi#qz_est_norm[i,1] - qz_est_norm[i+1,1]

    #R = funs.compute_optimal_rotation(traj[:time,:],thetas, scale=0)
    #theta = -np.pi/6 - np.pi
    #R = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    #R = np.array([[1, np.sin(theta)],[0, np.cos(theta)]])
    #R = np.array([[0, 1],[1, -1.73]])
    #qz_est_norm = qz_est_norm.dot(R)
    #plt.scatter(qz_est_norm[:,0],qz_est_norm[:,1])
    plt.figure()
    plt.plot(thetas_b[:,0],thetas_b[:,1],c="r",linewidth=.5)

In [ ]:
import copy
model_copy = copy.deepcopy(model)

def compute_poisson_loss_by_n(y, y_):
    return torch.mean(y_ - y * torch.log(y_ + 1e-9), dim=1)

_, _, z_, _, _ = model(torch.tensor(grid_cell_data.item()['y_train'], dtype=torch.float32, device=device)[trainer.neurons_train_ind], z=None)

## All weights to 0-1  
model_copy.ensemble_weights_train.data[:, 0] = -10**6
model_copy.ensemble_weights_train.data[:, 1] = 10**6
#llh_1 = torch.zeros(model_copy.ensemble_weights_train.size(0), device='cpu')

y_1, _, _, _, _ = model_copy(torch.tensor(grid_cell_data.item()['y_train'][grid_cell_data.item()['neurons_train_ind']], dtype=torch.float32, device=device), z=z_)
llh_train_train = compute_poisson_loss_by_n(
    torch.tensor(grid_cell_data.item()['y_train'][trainer.neurons_train_ind], dtype=torch.float32, device=device), y_1[:,:])
llh_1 = (llh_train_train.to(device='cpu'))

## All weights to 1-0
model_copy.ensemble_weights_train.data[:, 0] = 10**6
model_copy.ensemble_weights_train.data[:, 1] = -10**6
##llh_2 = torch.zeros(model_copy.ensemble_weights_train.size(0), device='cpu')

y_2, _, _, _, _ = model_copy(torch.tensor(grid_cell_data.item()['y_train'][grid_cell_data.item()['neurons_train_ind']], dtype=torch.float32, device=device), z=z_)
llh_train_train = compute_poisson_loss_by_n(
    torch.tensor(grid_cell_data.item()['y_train'][trainer.neurons_train_ind], dtype=torch.float32, device=device), y_2[:,:])

llh_2 = (llh_train_train.to(device='cpu'))

In [ ]:
## Plot greedy LLH
color_ens = ['#FF800E']*148 + ['#006BA4']*145
fig, ax = plt.subplots(figsize=(4,4))
ax.scatter(llh_1.detach().numpy(), llh_2.detach().numpy(), c=color_ens, s=15)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls='--', c='black')
ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
ax.set_xlabel('NLLH ens. 1', fontsize=20)
ax.set_ylabel('NLLH ens. 2', fontsize=20)
ax.set_yticks([0.25,0.5,0.75,1.00,1.25])
ax.set_xticks([0.25,0.5,0.75,1.00,1.25])
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
ax.grid()
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
fig.tight_layout()
#fig.savefig(time.strftime("./plots/%Y-%m-%d-") + "Fig5_ens_liks.pdf")

In [ ]:
## Load bins, tor. coord.s, ensemble weights and rfs
rec_spikes = rate_map_data.item()['binnedspikes']
y1_, y2_, z_, mu, logvar = model(torch.tensor(grid_cell_data.item()['y_train'], dtype=torch.float32, device=device)[trainer.neurons_train_ind], z=None)
z_ = z_.detach().cpu().numpy()
ensemble_weights_train = torch.nn.functional.softmax(model.ensemble_weights_train, dim=1).detach().cpu().numpy()
ensemble_weights_train2 = ensemble_weights_train.copy()
ensemble_weights_train2 = (ensemble_weights_train2 == ensemble_weights_train2.max(axis=1)[:,None]).astype(int)

ensemble_weights_test = torch.nn.functional.softmax(
    model.ensemble_weights_test, dim=1).detach().cpu().numpy()
ensemble_weights_test2 = ensemble_weights_test.copy()
ensemble_weights_test2 = (ensemble_weights_test2 == ensemble_weights_test2.max(axis=1)[:,None]).astype(int)
weights_full_2 = np.insert(ensemble_weights_train2, int(np.argwhere((grid_cell_data.item()['neurons_train_ind'])==False)[0,0]), ensemble_weights_test2[0,:], axis=0)

rfs = model.receptive_fields_train.detach().cpu().numpy()
rf_width = np.sqrt(np.exp(model.feature_basis.log_tuning_width.detach().cpu().numpy()))
rfs_full = np.insert(model.receptive_fields_train.detach().cpu().numpy(),
                     int(np.argwhere((grid_cell_data.item()['neurons_train_ind'])==False)[0,0]),
                     model.receptive_fields_test.detach().cpu().numpy()[0,:,:], axis=0)

In [ ]:
## Plot acc removed
true_weights = np.zeros(weights_full_2.shape)

true_weights[:149,0] = 1
true_weights[149:,1] = 1

acc1 = (true_weights == weights_full_2)
acc1 = np.all(acc1, axis=1)
acc1 = np.sum(acc1)/len(acc1)
print(acc1)
acc2 = (true_weights == 1-weights_full_2)
acc2 = np.all(acc2, axis=1)
acc2 = np.sum(acc2)/len(acc2)
print(acc2)
print("")

acc_list = []
for k in range(144):
  tot_mod1 = 149
  tot_mod2 = 145
  k_mod1 = tot_mod1 - k
  k_mod2 = 145 - k
  acc1_rem = np.delete((true_weights == weights_full_2), np.concatenate((np.arange(k_mod1, tot_mod1), np.arange(tot_mod1 + k_mod2, tot_mod1 + tot_mod2))), axis=0)
  acc1_rem = np.all(acc1_rem, axis=1)
  acc1_rem = np.sum(acc1_rem)/len(acc1_rem)
  #print(acc1_rem)
  acc2_rem = np.delete((true_weights == 1-weights_full_2), np.concatenate((np.arange(k_mod1, tot_mod1), np.arange(tot_mod1 + k_mod2, tot_mod1 + tot_mod2))), axis=0)
  acc2_rem = np.all(acc2_rem, axis=1)
  acc2_rem = np.sum(acc2_rem)/len(acc2_rem)
  #print(acc2_rem)
  acc = np.max([acc1_rem, acc2_rem])
  acc_list.append(acc)
  
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(acc_list, color='#006BA4')
#plt.ylim(0.5,1.01)
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
ax.set_xlabel('Neurons removed', fontsize=20)
ax.set_ylabel('Accuracy', fontsize=20)
ax.grid()
fig.tight_layout()
#fig.savefig(time.strftime("./plots/%Y-%m-%d-") + "Fig5_acc_removed.pdf")

In [ ]:
# Plot ensemble_weights
colors = np.array(['#006BA4', '#FF800E'])
test_weight = torch.nn.functional.softmax(
    model.ensemble_weights_test, dim=1).detach().cpu().numpy()
weights_full = np.insert(ensemble_weights_train, int(np.argwhere((grid_cell_data.item()['neurons_train_ind'])==False)[0,0]), test_weight[0,:], axis=0)
fig, ax = plt.subplots(1,1, figsize=(8,4))
n_list = np.arange(294) + 1
for i,col in enumerate(colors):
  ax.plot(n_list, weights_full[:,i], color=col)
#plt.legend(np.arange(num_ensemble))
#plt.title('Ensemble Weights')
#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)
ax.axvline(x=149.5, color='black', linestyle='--')
ax.set_xlabel('Neurons', fontsize=20)
ax.set_ylabel('Weights', fontsize=20)
ax.set_xticks([1, 50, 100, 150, 200, 250, 294])
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
ax.set_xlim(-2, 296)
ax.set_yticks([0,0.25,0.5,0.75,1.00])
#ax.set_ylim(0, 1)
ax.yaxis.grid(True)
fig.tight_layout()
fig.show()
#fig.savefig(time.strftime("./plots/%Y-%m-%d-") + "Fig5_weights_v3.pdf")

In [ ]:
## smooth images
def smooth_image(img, sigma):
  filterSize = max(np.shape(img))
  grid = np.arange(-filterSize+1, filterSize, 1)
  # covariance = np.square([sigma, sigma])
  xx,yy = np.meshgrid(grid, grid)

  pos = np.dstack((xx, yy))

  var = stats.multivariate_normal(mean=[0,0], cov=[[sigma**2,0],[0,sigma**2]])
  k = var.pdf(pos)
  k = k/np.sum(k)

  nans = np.isnan(img)
  imgA = img.copy()
  imgA[nans] = 0
  imgA = signal.convolve2d(imgA, k, mode='valid')
  imgD = img.copy()
  imgD[nans] = 0
  imgD[~nans] = 1
  radius = 1
  L = np.arange(-radius, radius + 1)
  X, Y = np.meshgrid(L, L)
  dk = np.array((X ** 2 + Y ** 2) <= radius ** 2, dtype=bool)
  imgE = np.zeros((filterSize+2,filterSize+2))
  imgE[1:-1,1:-1] = imgD
  imgE= binary_closing(imgE,iterations =1, structure =dk)
  imgD = imgE[1:-1,1:-1]

  imgB = np.divide(signal.convolve2d(imgD, k, mode='valid'), signal.convolve2d(np.ones(np.shape(imgD)), k, mode='valid'))
  imgC = np.divide(imgA,imgB)
  imgC[imgD==0] = -np.inf
  return imgC

In [ ]:
## Plot receptive fields

cnt = 0
box_bins = 50
torus_bins = 50
for j,i in enumerate(range(293)[::25]):
    if not (j%3):
      fig = plt.figure(figsize=(6,9))
      cnt += 1

    if i > 101:
      idx = i + 1
    else:
      idx = i
  
    plot_num_1 = (2*j)%6 + 1
    plot_num_2 = (2*j)%6 + 2
    ax = plt.subplot(3, 2, plot_num_1)
    bx = plt.subplot(3, 2, plot_num_2)
    
    box_field, _, _, _ = stats.binned_statistic_2d(rate_map_data.item()['x_pos'], rate_map_data.item()['y_pos'],
                                                   rec_spikes[idx,:], statistic = 'mean',
                                                   bins=box_bins, range=None, expand_binnumbers=True)  

    torus_field, _, _, _ = stats.binned_statistic_2d(z_[0, :, (1 - ensemble_weights_train2)[i, 0], 1]%(2*np.pi), z_[0, :, (1 - ensemble_weights_train2)[i, 0], 0]%(2*np.pi),
                                                     grid_cell_data.item()['y_train'][idx, :], statistic = 'mean',
                                                     bins=torus_bins, range=None, expand_binnumbers=True)

    ax.imshow(smooth_image(box_field,2.75), cmap='viridis')
    ax.set_xlim(0, box_bins-2)
    ax.set_ylim(0, box_bins-2)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    bx.imshow(smooth_image(torus_field, 2.75), cmap='viridis')
    t1 = (rfs[i, (1 - ensemble_weights_train2)[i, 0], 0]%(2*np.pi))*(torus_bins/(2*np.pi))
    t2 = (rfs[i, (1 - ensemble_weights_train2)[i, 0], 1]%(2*np.pi))*(torus_bins/(2*np.pi))
    bx.scatter(t1, t2, s=200, c='red', marker='o', edgecolor='black')
    circ_1 = Circle((t1, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_2 = Circle((t1, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    bx.add_patch(circ_1)
    bx.add_patch(circ_2)

    bx.scatter(t1-torus_bins, t2, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1+torus_bins, t2, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1, t2-torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1, t2+torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1-torus_bins, t2-torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1+torus_bins, t2-torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1-torus_bins, t2+torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1+torus_bins, t2+torus_bins, s=200, c='red', marker='o', edgecolor='black')

    circ_11 = Circle((t1-torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_21 = Circle((t1-torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_12 = Circle((t1+torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_22 = Circle((t1+torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_13 = Circle((t1, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_23 = Circle((t1, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_14 = Circle((t1, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_24 = Circle((t1, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_15 = Circle((t1-torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_25 = Circle((t1-torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_16 = Circle((t1+torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_26 = Circle((t1+torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_17 = Circle((t1-torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_27 = Circle((t1-torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_18 = Circle((t1+torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_28 = Circle((t1+torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')

    bx.add_patch(circ_11)
    bx.add_patch(circ_12)
    bx.add_patch(circ_13)
    bx.add_patch(circ_14)
    bx.add_patch(circ_15)
    bx.add_patch(circ_16)
    bx.add_patch(circ_17)
    bx.add_patch(circ_18)
    bx.add_patch(circ_21)
    bx.add_patch(circ_22)
    bx.add_patch(circ_23)
    bx.add_patch(circ_24)
    bx.add_patch(circ_25)
    bx.add_patch(circ_26)
    bx.add_patch(circ_27)
    bx.add_patch(circ_28)

    bx.set_xlim(0, torus_bins-2)
    bx.set_ylim(0, torus_bins-2)
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
    
    if plot_num_1 == 1:
      ax.set_title("Open Field", fontsize=20)
    if plot_num_2 == 2:
      bx.set_title("Toroidal", fontsize=20)
  
    #fig.savefig(time.strftime("./plots/%Y-%m-%d-") + "Fig5_" + str(cnt) + "_smooth_cont.pdf")

In [ ]:
## Plot receptive fields, appendix

cnt = 0
box_bins = 50
torus_bins = 50
rows = 22
cols = 14
for i in range(294):

    if i == 0 or i == 149:
      fig = plt.figure(figsize=(30,48))

    ax = plt.subplot(rows, cols, 2*(i%149) + 1)
    bx = plt.subplot(rows, cols, 2*(i%149) + 2)
    
    box_field, _, _, _ = stats.binned_statistic_2d(rate_map_data.item()['x_pos'], rate_map_data.item()['y_pos'],
                                                   rec_spikes[i,:], statistic = 'mean',
                                                   bins=box_bins, range=None, expand_binnumbers=True)  

    torus_field, _, _, _ = stats.binned_statistic_2d(z_[0, :, (1 - weights_full_2)[i, 0], 1]%(2*np.pi), z_[0, :, (1 - weights_full_2)[i, 0], 0]%(2*np.pi),
                                                     grid_cell_data.item()['y_train'][i, :], statistic = 'mean',
                                                     bins=torus_bins, range=None, expand_binnumbers=True)

    ax.imshow(smooth_image(box_field,2.75), cmap='viridis')
    ax.set_xlim(0, box_bins-2)
    ax.set_ylim(0, box_bins-2)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    bx.imshow(smooth_image(torus_field, 2.75), cmap='viridis')
    t1 = (rfs_full[i, (1 - weights_full_2)[i, 0], 0]%(2*np.pi))*(torus_bins/(2*np.pi))
    t2 = (rfs_full[i, (1 - weights_full_2)[i, 0], 1]%(2*np.pi))*(torus_bins/(2*np.pi))
    bx.scatter(t1, t2, s=200, c='red', marker='o', edgecolor='black')
    circ_1 = Circle((t1, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_2 = Circle((t1, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    bx.add_patch(circ_1)
    bx.add_patch(circ_2)

    bx.scatter(t1-torus_bins, t2, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1+torus_bins, t2, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1, t2-torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1, t2+torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1-torus_bins, t2-torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1+torus_bins, t2-torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1-torus_bins, t2+torus_bins, s=200, c='red', marker='o', edgecolor='black')
    bx.scatter(t1+torus_bins, t2+torus_bins, s=200, c='red', marker='o', edgecolor='black')

    circ_11 = Circle((t1-torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_21 = Circle((t1-torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_12 = Circle((t1+torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_22 = Circle((t1+torus_bins, t2), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_13 = Circle((t1, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_23 = Circle((t1, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_14 = Circle((t1, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_24 = Circle((t1, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_15 = Circle((t1-torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_25 = Circle((t1-torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_16 = Circle((t1+torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_26 = Circle((t1+torus_bins, t2-torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_17 = Circle((t1-torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_27 = Circle((t1-torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')
    circ_18 = Circle((t1+torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='black', lw=4, zorder=10, linestyle='-')
    circ_28 = Circle((t1+torus_bins, t2+torus_bins), 1*rf_width*(torus_bins/(2*np.pi)), facecolor='None', edgecolor='red', lw=2, zorder=11, linestyle='-')

    bx.add_patch(circ_11)
    bx.add_patch(circ_12)
    bx.add_patch(circ_13)
    bx.add_patch(circ_14)
    bx.add_patch(circ_15)
    bx.add_patch(circ_16)
    bx.add_patch(circ_17)
    bx.add_patch(circ_18)
    bx.add_patch(circ_21)
    bx.add_patch(circ_22)
    bx.add_patch(circ_23)
    bx.add_patch(circ_24)
    bx.add_patch(circ_25)
    bx.add_patch(circ_26)
    bx.add_patch(circ_27)
    bx.add_patch(circ_28)

    bx.set_xlim(0, torus_bins-2)
    bx.set_ylim(0, torus_bins-2)
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
    
    if (i%149) == 0 or (i%149) == 1 or (i%149) == 2 or (i%149) == 3 or (i%149) == 4 or (i%149) == 5 or (i%149) == 6:
      ax.set_title("Open Field", fontsize=20)
      bx.set_title("Toroidal", fontsize=20)
    
    if i == 293 or i == 148:
      fig.savefig(time.strftime("./plots/%Y-%m-%d-") + "All_rfs_" + str(i) + ".pdf")
      print("done")